In [5]:
import pandas as pd

# 读取原始总部城市数据
df_geo = pd.read_csv("E:/Documents/UCL/UCLminiproject/Group/world_com_city.csv")

# 去重：每家公司只保留一条总部记录
df_geo_hq = df_geo.drop_duplicates(subset=["company"])

# 保留总部城市、国家和经纬度字段
df_geo_hq_cleaned = df_geo_hq[["Parent_City_clean", "Country_head", "head_lat", "head_lng"]].dropna()

# 重命名列，便于后续合并
df_geo_hq_cleaned = df_geo_hq_cleaned.rename(columns={
    "Parent_City_clean": "city_clean",
    "Country_head": "iso2",
    "head_lat": "lat",
    "head_lng": "lng"
})

# 可选：查看结果前几行
print(df_geo_hq_cleaned.head())


         city_clean iso2      lat       lng
0     NEW BRUNSWICK   US  40.4870  -74.4450
588        SHANGHAI   CN  31.2286  121.4747
681           BASEL   CH  47.5606    7.5906
1069         RAHWAY   US  40.6077  -74.2807
1475       NEW YORK   US  40.6943  -73.9249


In [10]:
import pandas as pd

# 1. 读取原始文件（已标注功能分类的）
df = pd.read_excel("E:/Documents/UCL/UCLminiproject/Group/TOP2000_labeled_expert.xlsx")  # 替换为你的文件名

# 2. 确保销售额是数字
df["Business line - Sales\nth USD"] = pd.to_numeric(df["Business line - Sales\nth USD"], errors="coerce")

# 3. 汇总销售额（按公司 + 功能类别）
sales_sum = (
    df[df["function_category"].isin(["RS", "PS", "SS"])]
    .groupby(["Company name Latin alphabet", "function_category"])["Business line - Sales\nth USD"]
    .sum()
    .unstack(fill_value=0)
    .rename(columns={"RS": "RS_sales", "PS": "PS_sales", "SS": "SS_sales"})
    .reset_index()
)

# 4. 合并回原始数据（保留所有列，按公司名合并）
df_with_sales = df.merge(sales_sum, how="left", on="Company name Latin alphabet")

# 5. 查看结果
df_with_sales.head()


,Unnamed: 0,Company name Latin alphabet,Country ISO code,City\nLatin Alphabet,"NACE Rev. 2, core code (4 digits)",Operating revenue (Turnover)\nth USD Last avail. yr,Number of employees\nLast avail. yr,Profit margin\nLast avail. yr,MSCI - Overall indicative ESG score\nLast avail. yr,MSCI - Environmental pillar indicative score\nLast avail. yr,MSCI - Social pillar indicative score\nLast avail. yr,MSCI - Governance pillar indicative score\nLast avail. yr,Business line - Sales\nth USD,label,function_category,PS_sales,RS_sales,SS_sales
0,1.0,JOHNSON & JOHNSON,US,NEW BRUNSWICK,2120.0,88821000.0,138100,18.787,3.5,1.4,3.2,4.6,56965000.0,innovative medicine,RS,0.0,56965000.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31857000.0,medtech,PS,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88822000.0,segment total,OTHER,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88822000.0,consolidated total,OTHER,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14574000.0,innovative medicine,RS,NaN,NaN,NaN


In [12]:
!pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.1 MB 7.2 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.1 MB 7.5 MB/s eta 0:00:02
   ----------------- ---------------------- 4.7/11.1 MB 8.1 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.1 MB 7.9 MB/s eta 0:00:01
   ---------------------------- ----------- 7.9/11.1 MB 8.1 MB/s eta 0:00:01
   ----------------------------------- ---- 9.7/11.1 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 7.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Step 1: 读取数据
df = pd.read_excel("E:/Documents/UCL/UCLminiproject/Group/TOP2000_corrected_ffill_sales.xlsx")  # 替换为你的文件路径

# Step 2: 选择构建韧性指数所需的字段
cols = {
    "Operating revenue (Turnover)\nth USD Last avail. yr": "scale",
    "MSCI - Overall indicative ESG score\nLast avail. yr": "esg_total",
    "MSCI - Environmental pillar indicative score\nLast avail. yr": "esg_env",
    "MSCI - Social pillar indicative score\nLast avail. yr": "esg_soc",
    "MSCI - Governance pillar indicative score\nLast avail. yr": "esg_gov",
    "functional_diversity": "func_div"
}
df_renamed = df[list(cols.keys())].rename(columns=cols)

# Step 3: 删除缺失值（可选）
for col in df_renamed.columns:
    df_renamed[col] = pd.to_numeric(df_renamed[col], errors="coerce")

df_clean = df_renamed.dropna()

# Step 4: Min-Max 归一化
scaler = MinMaxScaler()
normalized = scaler.fit_transform(df_clean)
df_scaled = pd.DataFrame(normalized, columns=df_clean.columns)

# Step 5: 计算韧性指数（等权平均）
df_scaled["resilience_index"] = df_scaled.mean(axis=1)

# Step 6: 合并回原始公司数据
df_result = df.copy()
df_result["resilience_index"] = None
df_result.loc[df_clean.index, "resilience_index"] = df_scaled["resilience_index"].values

# Step 7: 保存为新文件
df_result.to_excel("TOP2000_with_resilience_index.xlsx", index=False)


In [9]:
import pandas as pd

# 读取 TOP2000 数据
df_companies = pd.read_excel("E:/Documents/UCL/UCLminiproject/Group/TOP2000_with_resilience_index.xlsx")

# 去除空城市和国家代码行
df_companies = df_companies.dropna(subset=["City", "ISO"])

# 清洗城市和国家字段
df_companies["city_cleaned"] = (
    df_companies["City"]
    .str.strip()
    .str.upper()
    .str.replace(r",.*$", "", regex=True)
    .str.replace(r'\b(KU|SHI|CITY|SI|DISTRICT)\b', '', regex=True)
    .str.replace(r'\s{2,}', ' ', regex=True)
    .str.strip()
)

df_companies["ISO"] = df_companies["ISO"].str.strip().str.upper()

# ✅ 匹配总部经纬度
df_merged = pd.merge(
    df_companies,
    df_geo_hq_cleaned,
    left_on=["city_cleaned", "ISO"],
    right_on=["city_clean", "iso2"],
    how="left"
)

# 查看匹配情况
print("总记录数：", len(df_merged))
print("未匹配经纬度的记录数：", df_merged['lat'].isna().sum())
print("已匹配经纬度的记录数：", df_merged['lat'].notna().sum())

# 可选：保存结果
df_merged.to_csv("TOP2000_with_coords.csv", index=False)


总记录数： 4587
未匹配经纬度的记录数： 79
已匹配经纬度的记录数： 4508


In [6]:
unmatched = df_companies[~df_companies['city'].isin(df_geo['city_clean'])]
print(unmatched['city'].value_counts().head(20))


city
MILANO                    23
MINATO KU                 12
WIEN                       9
CHIYODA KU                 9
CHUO KU                    7
GRAND CAYMAN               6
HANGZHOU CITY              6
DUBLIN 2                   5
OSAKA SHI CHUO KU          5
MUENCHEN                   5
BRUXELLES                  4
WUXI CITY                  4
SUZHOU CITY                4
YONGIN-SI                  4
TAIZHOU CITY, ZHEJIANG     4
TOYAMA SHI                 4
JAKARTA, JAVA              3
PUURS-SINT-AMANDS          3
SHIJIAZHUANG CITY          3
GWACHEON-SI                3
Name: count, dtype: int64


In [2]:
import geopandas as gpd
from shapely.geometry import Point

# 读取点数据
gdf = gpd.read_file("E:/Downloads/City_level_resilience_data_with_clusters.geojson")

# 创建缓冲polygon，单位是度（建议试 0.1 ~ 0.5 km 的转换）
gdf['geometry'] = gdf['geometry'].buffer(0.5)  # 0.1度，大概11km半径

# 保存为新的polygon geojson
gdf.to_file("E:/Downloads/City_level_resilience_data_polygons.geojson", driver='GeoJSON')


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27500\892933815.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf['geometry'].buffer(0.5)  # 0.1度，大概11km半径


In [2]:
import geopandas as gpd

# 直接读取 geojson
gdf = gpd.read_file('E:/Documents/UCL/UCLminiproject/demo/City_level_resilience_data_with_clusters.geojson')

# 提取经纬度
coords = gdf.geometry.apply(lambda geom: (geom.x, geom.y))

# 检查重复
duplicates = coords.value_counts()
print(duplicates[duplicates > 1])


geometry
(110.3488, 20.0186)    2
(-81.3811, 19.295)     2
(117.2273, 31.8206)    2
(117.0207, 36.6702)    2
(91.1719, 29.6534)     2
(125.326, 43.897)      2
(112.939, 28.228)      2
(126.6409, 45.7576)    2
(118.7789, 32.0608)    2
(121.6245, 29.8603)    2
(126.9833, 37.5667)    2
(116.964, 33.648)      2
(-79.3733, 43.7417)    2
Name: count, dtype: int64


In [3]:
import pandas as pd
import geopandas as gpd

# 读取 geojson
gdf = gpd.read_file('E:/Documents/UCL/UCLminiproject/demo/City_level_resilience_data_with_clusters.geojson')

# 提取经纬度
gdf['coords'] = gdf['geometry'].apply(lambda x: (round(x.x, 4), round(x.y, 4)))  # 保留4位小数防止浮点误差

# 找到重复的坐标
duplicates = gdf[gdf.duplicated(subset='coords', keep=False)]

# 查看相同坐标下有哪些City
grouped = duplicates.groupby('coords')['City'].unique()

print(grouped)


coords
(-81.3811, 19.295)         [GEORGE TOWN, George Town]
(-79.3733, 43.7417)                [TORONTO, Toronto]
(91.1719, 29.6534)              [LHASA, LHASA, TIBET]
(110.3488, 20.0186)          [HAIKOU, HAIKOU, HAINAN]
(112.939, 28.228)         [CHANGSHA, CHANGSHA, HUNAN]
(116.964, 33.648)           [SUZHOU, SUZHOU, JIANGSU]
(117.0207, 36.6702)          [JINAN, JINAN, SHANDONG]
(117.2273, 31.8206)             [HEFEI, HEFEI, ANHUI]
(118.7789, 32.0608)       [NANJING, NANJING, JIANGSU]
(121.6245, 29.8603)        [NINGBO, NINGBO, ZHEJIANG]
(125.326, 43.897)       [CHANGCHUN, CHANGCHUN, JILIN]
(126.6409, 45.7576)    [HARBIN, HARBIN, HEILONGJIANG]
(126.9833, 37.5667)                    [SEOUL, Seoul]
Name: City, dtype: object
